### Imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Load the beta-VAE module
import bvae_model as bvae
import spectrum_preprocessing as sp

VAE loaded successfully!


### Load raw spectra

In [9]:
#your path
path = "C:/Users/Zach/Documents/Lumos-II-Image-Processing/spectral_data/2025-09-17 Zach SMP65 010 90C SR 42d 520um.csv"

In [10]:
filename = path.split("/")[-1]
filename

'2025-09-17 Zach SMP65 010 90C SR 42d 520um.csv'

In [11]:
df = pd.read_csv(path, skiprows=[1,2])
print(f"Dataframe shape: {df.shape}")

Dataframe shape: (6144, 2099)


In [12]:
df

,map_x,map_y,Sample Name,3997.9328286151754,3996.348861567388,3994.7648945196006,3993.1809274718134,3991.5969604240263,3990.0129933762387,3988.4290263284515,...,693.7775669308426,692.1935998830554,690.6096328352678,689.0256657874806,687.4416987396935,685.8577316919059,684.2737646441187,682.6897975963311,681.105830548544,679.5218635007568
0,40500.2,23850.5,2025-07-21 Zach SMP65 010 90C 42d 500um,0.173777,0.173847,0.177223,0.181519,0.183783,0.183929,0.183430,...,-0.459046,-0.409898,-0.340203,-0.266194,-0.241650,-0.219905,-0.197531,-0.367240,-0.248315,-0.163803
1,40505.6,23850.5,2025-07-21 Zach SMP65 010 90C 42d 500um,0.182412,0.182180,0.183009,0.184252,0.185426,0.186737,0.187988,...,0.464279,0.455207,0.520533,0.496881,0.418492,0.379761,0.401518,0.469424,0.543619,0.529311
2,40511.1,23850.5,2025-07-21 Zach SMP65 010 90C 42d 500um,0.173305,0.175676,0.177717,0.177642,0.176736,0.177838,0.181368,...,0.020502,0.140653,0.302099,0.405964,0.447103,0.468335,0.494399,0.458751,0.515179,0.626890
3,40516.5,23850.5,2025-07-21 Zach SMP65 010 90C 42d 500um,0.210148,0.213458,0.216452,0.217675,0.218557,0.220930,0.224773,...,-0.470702,-0.436814,-0.249563,-0.110681,-0.288037,-0.561324,-0.413342,-0.185088,-0.179641,-0.306619
4,40521.9,23850.5,2025-07-21 Zach SMP65 010 90C 42d 500um,0.222067,0.221189,0.220800,0.221444,0.222824,0.224614,0.226496,...,-0.201703,-0.414454,-0.650868,-0.396083,-0.371536,-0.226905,-0.028248,-0.015734,-0.052728,-0.155342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,40820.8,24365.6,2025-07-21 Zach SMP65 010 90C 42d 500um,0.145345,0.146758,0.149381,0.152688,0.154619,0.155009,0.155888,...,-0.311145,-0.287159,-0.352346,-0.453550,-0.501523,-0.513754,-0.577307,-0.650945,-0.587705,-0.567465
6140,40826.2,24365.6,2025-07-21 Zach SMP65 010 90C 42d 500um,0.154998,0.155018,0.154622,0.154178,0.153920,0.154405,0.156113,...,-0.221203,-0.258107,-0.357979,-0.536340,-0.659724,-0.735318,-0.726095,-0.628477,-0.498093,-0.411662
6141,40831.6,24365.6,2025-07-21 Zach SMP65 010 90C 42d 500um,0.168808,0.169854,0.172368,0.175288,0.177274,0.178065,0.178377,...,-0.448509,-0.469105,-0.368630,-0.280722,-0.252272,-0.246988,-0.248498,-0.240105,-0.245191,-0.262622
6142,40837.1,24365.6,2025-07-21 Zach SMP65 010 90C 42d 500um,0.179617,0.177724,0.177280,0.178185,0.179158,0.180153,0.182013,...,-0.271185,-0.355372,-0.426378,-0.449514,-0.416545,-0.516891,-0.712586,-0.831816,-0.807714,-0.620569


# Sort the wavenumber columns and data and put in a dataframe
- necessary for input to pipeline below

In [13]:
wavenumber = df.columns.values[4:].astype(float)
wavenumber.sort()
wavenumber = wavenumber.astype(str)
spectra_df = df[wavenumber].copy()
np.dtype(spectra_df.columns.values.astype(float)[0])

dtype('float64')

### Run the pipeline
The `bvae.pipeline()` returns the interpolated wavenumber array; the interpolated absorbance array; the beta-model encodings L1 L2 and L3; the reconstruction MSE; and the reconstructed spectrum.

In [15]:
L1_list=[]
L2_list=[]
L3_list=[]
MSE_list=[]
wavenumber_list=[]
absorbance_list=[]
reconstructed_list=[]
for i in range(spectra_df.shape[0]):    
    f = spectra_df.columns.values[:].astype('float32')
    a = spectra_df.iloc[i,:].values
    try:
        wavenumber, absorbance, encodings, mse, reconstructed = bvae.pipeline(f, a)
        L1 = encodings[0]
        L2 = encodings[1]
        L3 = encodings[2]
    except Exception as e:
        print(e)
        L1 = 0
        L2 = 0
        L3 = 0
        mse = 100
        reconstructed = np.zeros(len(wavenumber))
        
    L1_list.append(L1)
    L2_list.append(L2)
    L3_list.append(L3)
    MSE_list.append(mse)
    absorbance_list.append(absorbance)
    reconstructed_list.append(reconstructed)
    
    #print(encodings)
    #print(mse)
    #plt.plot(absorbance,c='b')
    #plt.plot(reconstructed,c='r')
    #plt.show()

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * [<tf.Tensor 'inputs:0' shape=(1063,) dtype=float32>]
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 505), dtype=tf.float32, name='input_1')
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 505), dtype=tf.float32, name='inputs')
    * False
  Keyword arguments: {}

Option 3:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 505), dtype=tf.float32, name='inputs')
    * True
  Keyword arguments: {}

Option 4:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 505), dtype=tf.float32, name='input_1')
    * True
  Keyword arguments: {}


NameError: name 'absorbance' is not defined

# Save results to dataframe

In [ ]:
hyperspectrum =pd.DataFrame(absorbance_list)
hyperspectrum.columns = wavenumber
hyperspectrum['map_x'] = df['map_x'].values
hyperspectrum['map_y'] = df['map_y'].values

In [ ]:
reconstructed =pd.DataFrame(reconstructed_list)
reconstructed.columns = wavenumber
reconstructed['map_x'] = df['map_x'].values
reconstructed['map_y'] = df['map_y'].values

In [ ]:
bvae_df =pd.DataFrame()
bvae_df['map_x'] = df['map_x'].values
bvae_df['map_y'] = df['map_y'].values
bvae_df['L1'] = L1_list
bvae_df['L2'] = L2_list
bvae_df['L3'] = L3_list
bvae_df['MSE'] = MSE_list

# Save to csv files for viewing in Quasar

In [ ]:
path_to_save = 'C:/Users/Zach/Documents/BVAE_processed/'

bvae_df.to_csv(path_to_save +'bvae_'+filename)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Hyperspectra/"
file=os.path.join(path_to_save + 'hyperspectrum_'+filename)
hyperspectrum.to_csv(file, index=False)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Reconstructions/"
reconstructed.to_csv(path_to_save +'reconstructed_'+filename, index=False)